In [1]:
import requests
import numpy as np
import pandas as pd
import os

# Embeddings

In [33]:
# def generateEmbeddings(text):
    
#     header = {'Content-Type': 'application/json'}
#     url ="http://localhost:8088/?task=embeddings&prompt=" + text.replace(" ","%20")
#     req = requests.get(url,headers=header)
#     r = dict(req.json())
    
#     return r

def generateEmbeddings(text):

    myobj = {'model': 'instruct','task': 'embeddings','instruction':'', 'input': text}
   
    header = {'Content-Type': 'application/json'}
    url = "http://localhost:8088/" 

    x = requests.post(url, json = myobj, headers=header)
    r = dict(x.json())
    
    return r

In [34]:

def generateEmbeddingsAverage(text):
    
    r = generateEmbeddings(text)
    
    ret = np.mean(np.array(list(r.values())), axis=0)
    
    return ret

## Translate SHAC into embeddings

In [35]:
import sys

In [36]:
sys.path.insert(0, "..")

In [37]:
from src.process_SHAC import load_process_SHAC

In [38]:
import joblib

In [39]:
from tqdm import tqdm


from tqdm.notebook import tqdm

In [40]:
df = load_process_SHAC(replaceNA="all")

In [41]:
df

,id,Drug,Alcohol,Tobacco,SubstanceAgg,set,location,text
0,3407,True,False,False,True,train,uw,Social History: Work: Unemployed Lives with m...
1,3313,False,True,False,True,train,uw,SOCIAL HISTORY: Living situation: Comfortabl...
2,3045,True,False,True,True,train,uw,HABITS: Tobacco Use: Approximately 20 pack-ye...
3,4399,False,True,True,True,train,uw,Social History: Immigrated from [LOCATION] in ...
4,3637,False,False,True,True,train,uw,HABITS: Tobacco Use: Formerly smoked 1-2 ciga...
...,...,...,...,...,...,...,...,...
4400,2884,True,True,False,True,test,mimic,Social History: lives w/ mother in [**Name2 (N...
4401,2728,False,False,True,True,test,mimic,Social History: lives with wife (recent nursin...
4402,2854,False,True,True,True,test,mimic,"Social History: quit smoking several yrs ago, ..."
4403,2726,False,False,False,False,test,mimic,"Social History: Denies Tob, EtOH, or Illicit d..."


In [42]:
tqdm.pandas()

# Store All Embeddings

In [ ]:
df['LLaMaEmbeddings'] = df.progress_apply(lambda x: 
                                         generateEmbeddings(x['text'].replace("&"," and ")), 
                                         axis=1)

In [ ]:
df.to_pickle("../output/LLaMaEmbeddings/df_SHAC.pkl")

In [ ]:
# with open("/edata/CohenLybarger/xiruo_Project/cognitive_distortions_Transfer_Learning_GitLab/data/LLaMaEmbeddings/df_avh.pkl", 'wb') as f:
#     joblib.dump(value=df_avh, filename=f)

# Store Averaged Embeddings

In [ ]:
# out_file = "../output/LLaMaEmbeddings/df_Average_SHAC.csv"
# for idx, row in df.iterrows():
#     if idx == 3:
#         break
        
#     row['embedding'] = generateEmbeddingsAverage(row['text'])
#     row.to_frame().T.to_csv(out_file, index=False, mode='a', header=not os.path.exists(out_file))
#     # df.iloc[[idx]].to_csv(out_file, index=False, mode='a', header=not os.path.exists(out_file))
#     # tmp = df.iloc[[idx]].copy()
#     # tmp['embedding'] = generateEmbeddingsAverage(df.iloc[idx]['text'])





In [12]:
df['LLaMaEmbeddings'] = df.progress_apply(lambda x: 
                                         generateEmbeddingsAverage(x['text'].replace("&"," and ")), 
                                         axis=1)

df.to_pickle("../output/LLaMaEmbeddings/df_Average_SHAC.pkl")

  0%|          | 0/4405 [00:00<?, ?it/s]

# Generations

In [18]:
r = requests.get(url, headers=header)
sresp = ""
for i in r.iter_lines():
    sresp = sresp + i.decode() #replace(‘\\xe2\\x80\\x99’,“\’“)

In [20]:
r

<Response [200]>

In [25]:
def complete(prompt, chars=32):
    header = {"Content-Type": "application/json"}
    url ="http://localhost:8088/?task=generate:"+str(chars)+"&prompt="+prompt.replace(" ","%20")
    r = requests.get(url,headers=header)
    sresp = ""
    for i in r.iter_lines():
        sresp = sresp + i.decode()#replace(‘\\xe2\\x80\\x99’,“\’“)
    return sresp

In [28]:
complete("what is LLaMa by Meta? ", chars=100)

' what is LLaMa by Meta? 24/11/2016 76466465LLaMa, a new initiative by Meta, is a web-based community aimed at bringing together stakeholders in the fields of Language Technologies and Machine Translation. The main objective of LLaMa will be to facilitate research and development across MT and Language Technologies. It aims to do so by connecting people'